In [1]:
import pandas as pd
from sqlalchemy import create_engine
import datetime
# from config import password

# STEP 1: EXTRACT

In [2]:
# DATASOURCE 1: Extract all records from winemag.csv - source kaggle.com

wine_file = "Resources/winemag.csv"
df_wine = pd.read_csv(wine_file)

print("Total records ", df_wine['winery'].count())
df_wine.head()

Total records  129971


,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [3]:
# Only keep required columns and rename them, change US to United States and fill region with a value to allow aggregate function later
wine_cols = ['Unnamed: 0','country','province','region_1','title','variety','winery','points','price']
df_wine_base = df_wine[wine_cols].copy()

df_wine_base = df_wine_base.rename(columns={'Unnamed: 0':'wine_id',
                                            'country': 'country_name',
                                            'province': 'province_name',
                                            'region_1':'region_name',
                                            'title':'wine_name',
                                            'variety':'wine_type',
                                            'winery':'winery_name',
                                            'points':'rating',
                                            'price':'price'})

df_wine_base = df_wine_base.replace(to_replace='US', value='United States')
df_wine_base['region_name'].fillna("No region", inplace = True)
df_wine_base.head()

,wine_id,country_name,province_name,region_name,wine_name,wine_type,winery_name,rating,price
0,0,Italy,Sicily & Sardinia,Etna,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,87,NaN
1,1,Portugal,Douro,No region,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,87,15.0
2,2,United States,Oregon,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,87,14.0
3,3,United States,Michigan,Lake Michigan Shore,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,87,13.0
4,4,United States,Oregon,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,87,65.0


In [4]:
# Drop any records that have null values in any of their column fields
print("Before dropping null values: ") 
print(df_wine_base.count())

df_wine_base = df_wine_base.dropna(how='any')

print("After dropping null values: ") 
print(df_wine_base.count())

Before dropping null values: 
wine_id          129971
country_name     129908
province_name    129908
region_name      129971
wine_name        129971
wine_type        129970
winery_name      129971
rating           129971
price            120975
dtype: int64
After dropping null values: 
wine_id          120915
country_name     120915
province_name    120915
region_name      120915
wine_name        120915
wine_type        120915
winery_name      120915
rating           120915
price            120915
dtype: int64


In [5]:
# Group df_wine_base and calculate average price and rating per unique wine in the database)
df_wine_data = df_wine_base.groupby(['wine_id' ,
                                     'country_name', 
                                     'province_name', 
                                     'region_name',
                                     'wine_name',
                                     'wine_type',
                                     'winery_name'], as_index=False)['rating','price'].mean().copy()

# df_wine_base.reset_index(inplace=True)
print("Total records ", df_wine_data['country_name'].count())

df_wine_data.head()

Total records  120915


,wine_id,country_name,province_name,region_name,wine_name,wine_type,winery_name,rating,price
0,1,Portugal,Douro,No region,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,87,15.0
1,2,United States,Oregon,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,87,14.0
2,3,United States,Michigan,Lake Michigan Shore,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,87,13.0
3,4,United States,Oregon,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,87,65.0
4,5,Spain,Northern Spain,Navarra,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,87,15.0


In [6]:
# DATASOURCE 2: extract country information from countries.csv - source: wikipedia
    
country_file = "Resources/countries.csv"
df_countries = pd.read_csv(country_file)
df_countries.head()
df_countries.reset_index(inplace=True)
df_countries = df_countries.rename(columns={'index':'country_id',
                                            'country':'country_abbr',
                                            'name': 'country_name'})
df_countries = df_countries.dropna(how='any')
print("Total records: ", df_countries['country_id'].count())
      
df_countries.head()

Total records:  243


,country_id,country_abbr,latitude,longitude,country_name
0,0,AD,42.546245,1.601554,Andorra
1,1,AE,23.424076,53.847818,United Arab Emirates
2,2,AF,33.939110,67.709953,Afghanistan
3,3,AG,17.060816,-61.796428,Antigua and Barbuda
4,4,AI,18.220554,-63.068615,Anguilla


In [7]:
# DATASOURCE 3: Directly load the temperatures by country table of the internet - source: Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_average_yearly_temperature'
tables = pd.read_html(url)

df_temps = pd.DataFrame(tables[0])

# Rename columns
df_temps = df_temps.rename(columns={'Country':'country_name','Average yearly temperature (1961–1990, degrees Celsius)':'avg_temp'})

# # Reset index for fast lookup by df_countries
df_temps = df_temps.set_index('country_name')

print("Total records: ", df_temps['avg_temp'].count())
df_temps.head()

Total records:  191


,avg_temp
country_name,
Burkina Faso,28.25
Mali,28.25
Kiribati,28.20
Djibouti,28.00
Tuvalu,28.00


In [8]:
# DATASOURCE 3b: Directly load the temperatures by country table of the internet - source: www.currentresults.com
state_temps_file = "Resources/state_temps.csv"
df_states = pd.read_csv(state_temps_file)
df_states.head()

df_states = df_states.rename(columns={'State':'state',
                                            'Avg °F':'avg_F',
                                            'Avg °C': 'avg_C'})

print("Total records: ", df_states.state.count())

df_states_temps = df_states.set_index('state').copy()
df_states_temps.head()

Total records:  50


,avg_F,avg_C
state,,
Alabama,62.8,17.1
Alaska,26.6,-3.0
Arizona,60.3,15.7
Arkansas,60.4,15.8
California,59.4,15.2


# STEP 2: TRANSFORM

In [9]:
# Add temperature to df_countries when found
df_countries['temperature'] = ""
list_temp = []

for index, row in df_countries.iterrows():
    try:
        list_temp = df_temps.loc[str(row[4]),'avg_temp']
        df_countries.iloc[index, 5] = list_temp
    except:
        pass

# Two dataframes - one for saving to the database later that includes the index_id and the other for lookup for df_wine_data
df_countries.head()
df_countries_lookup = df_countries.copy()
df_countries_lookup = df_countries_lookup.set_index("country_name")
df_countries_lookup.head()

,country_id,country_abbr,latitude,longitude,temperature
country_name,,,,,
Andorra,0,AD,42.546245,1.601554,07.60
United Arab Emirates,1,AE,23.424076,53.847818,27.00
Afghanistan,2,AF,33.939110,67.709953,12.60
Antigua and Barbuda,3,AG,17.060816,-61.796428,26.00
Anguilla,4,AI,18.220554,-63.068615,


In [10]:
# from df_wine_base, create seperate tables, to normalize the database
# df_wineries, df_provinces and df_regions. df_countries was created by pulling so called 'master data' of the internet.

# For the wineries

df_winery_base = df_wine_base.groupby(['winery_name']).count()
df_winery_base.reset_index(inplace=True)
df_wineries = df_winery_base[['winery_name']]
df_wineries.reset_index(inplace=True)
df_wineries = df_wineries.rename(columns={'index':'winery_id'})
print("Total records: ", df_wineries['winery_id'].count())

# Two dataframes - one for saving to the database later that includes the index_id and the other for lookup for df_wine_data
df_wineries.head()
df_wineries_lookup = df_wineries.copy()
df_wineries_lookup = df_wineries_lookup.set_index("winery_name")
df_wineries_lookup.head()

Total records:  15843


,winery_id
winery_name,
1+1=3,0
10 Knots,1
100 Percent Wine,2
1000 Stories,3
1070 Green,4


In [11]:
# For the provinces

df_provinces_base = df_wine_base.groupby(['province_name']).count()
df_provinces_base.reset_index(inplace=True)
df_provinces = df_provinces_base[['province_name']]
df_provinces.reset_index(inplace=True)
df_provinces = df_provinces.rename(columns={'index':'province_id'})
print("Total records: ", df_provinces['province_id'].count())

# Two dataframes - one for saving to the database later that includes the index_id and the other for lookup for df_wine_data
df_provinces.head()
df_provinces_lookup = df_provinces.copy()
df_provinces_lookup = df_provinces_lookup.set_index("province_name")
df_provinces_lookup.head()

Total records:  422


,province_id
province_name,
Achaia,0
Aconcagua Costa,1
Aconcagua Valley,2
Aegean,3
Agioritikos,4


In [12]:
df_states_temps.head()

,avg_F,avg_C
state,,
Alabama,62.8,17.1
Alaska,26.6,-3.0
Arizona,60.3,15.7
Arkansas,60.4,15.8
California,59.4,15.2


In [13]:
# Add temperature to df_provinces when found
df_provinces['avg_f'] = ""
df_provinces['avg_c'] = ""
temp_C = []
temp_F = []

for index, row in df_provinces.iterrows():
    try:
        temp_F = df_states_temps.loc[str(row[1]),'avg_F']
        df_provinces.iloc[index, 2] = temp_F
#         print(row)
        temp_C = df_states_temps.loc[str(row[1]),'avg_C']
        df_provinces.iloc[index, 3] = temp_C
    except:
        pass

df_provinces.loc[df_provinces['province_name'] == 'Missouri']

,province_id,province_name,avg_f,avg_c
219,219,Missouri,54.5,12.5


In [14]:
# For the regions

df_regions_base = df_wine_base.groupby(['region_name']).count()
df_regions_base.reset_index(inplace=True)
df_regions = df_regions_base[['region_name']]
df_regions.reset_index(inplace=True)
df_regions = df_regions.rename(columns={'index':'region_id'})
print("Total records: ", df_regions['region_id'].count())

# Two dataframes - one for saving to the database later that includes the index_id and the other for lookup for df_wine_data
df_regions.head()
df_regions_lookup = df_regions.copy()
df_regions_lookup = df_regions_lookup.set_index("region_name")
df_regions_lookup.head()

Total records:  1205


,region_id
region_name,
Abruzzo,0
Adelaida District,1
Adelaide,2
Adelaide Hills,3
Adelaide Plains,4


In [15]:
# For the wine_types

df_wine_types_base = df_wine_base.groupby(['wine_type']).count()
df_wine_types_base.reset_index(inplace=True)
df_wine_types = df_wine_types_base[['wine_type']]
df_wine_types.reset_index(inplace=True)
df_wine_types = df_wine_types.rename(columns={'index':'wine_type_id'})
print("Total records: ", df_wine_types['wine_type_id'].count())

# Two dataframes - one for saving to the database later that includes the index_id and the other for lookup for df_wine_data
df_wine_types.head()
# df_wine_types_lookup = df_wine_types.copy()
# df_wine_types_lookup = df_wine_types_lookup.set_index("wine_type")
# df_wine_types_lookup.head()

Total records:  691


,wine_type_id,wine_type
0,0,Abouriou
1,1,Agiorgitiko
2,2,Aglianico
3,3,Aidani
4,4,Airen


In [16]:
# For reference for the part below: df_wine_base before
df_wine_data.head()

,wine_id,country_name,province_name,region_name,wine_name,wine_type,winery_name,rating,price
0,1,Portugal,Douro,No region,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,87,15.0
1,2,United States,Oregon,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,87,14.0
2,3,United States,Michigan,Lake Michigan Shore,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,87,13.0
3,4,United States,Oregon,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,87,65.0
4,5,Spain,Northern Spain,Navarra,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,87,15.0


In [17]:
# Now replace all _name strings in df_wine_base with their correlating _id keys
print("Start time: ", datetime.datetime.now().time())
# limit = 3

df_wine_data = df_wine_data.merge(df_countries, on='country_name', how='inner')
df_wine_data = df_wine_data.merge(df_provinces, on='province_name', how='inner')
df_wine_data = df_wine_data.merge(df_regions, on='region_name', how='inner')
df_wine_data = df_wine_data.merge(df_wineries, on='winery_name', how='inner')
df_wine_data = df_wine_data.merge(df_wine_types, on='wine_type', how='inner')
    
print("Finish time: ", datetime.datetime.now().time())
df_wine_data.head()

Start time:  11:16:24.159653
Finish time:  11:16:24.569134


,wine_id,country_name,province_name,region_name,wine_name,wine_type,winery_name,rating,price,country_id,country_abbr,latitude,longitude,temperature,province_id,avg_f,avg_c,region_id,winery_id,wine_type_id
0,1,Portugal,Douro,No region,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,87,15.0,180,PT,39.399872,-8.224454,,107,,,751,12189,442
1,1655,Portugal,Douro,No region,Quinta dos Avidagos 2015 Lote 138 Red (Douro),Portuguese Red,Quinta dos Avidagos,85,11.0,180,PT,39.399872,-8.224454,,107,,,751,12189,442
2,4840,Portugal,Douro,No region,Quinta dos Avidagos 2014 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,87,15.0,180,PT,39.399872,-8.224454,,107,,,751,12189,442
3,22847,Portugal,Douro,No region,Quinta dos Avidagos 2011 Lenuma Red (Douro),Portuguese Red,Quinta dos Avidagos,93,65.0,180,PT,39.399872,-8.224454,,107,,,751,12189,442
4,70711,Portugal,Douro,No region,Quinta dos Avidagos 2012 Premium Red (Douro),Portuguese Red,Quinta dos Avidagos,87,29.0,180,PT,39.399872,-8.224454,,107,,,751,12189,442


In [18]:
wine_cols = ['wine_id','wine_name','country_id','province_id','region_id','winery_id','wine_type_id','rating','price']
df_wine_final = df_wine_data[wine_cols].copy()

# df_wine_final = df_wine_final.rename(columns={'index':'wine_id'})

In [19]:
df_wine_final.head()

,wine_id,wine_name,country_id,province_id,region_id,winery_id,wine_type_id,rating,price
0,1,Quinta dos Avidagos 2011 Avidagos Red (Douro),180,107,751,12189,442,87,15.0
1,1655,Quinta dos Avidagos 2015 Lote 138 Red (Douro),180,107,751,12189,442,85,11.0
2,4840,Quinta dos Avidagos 2014 Avidagos Red (Douro),180,107,751,12189,442,87,15.0
3,22847,Quinta dos Avidagos 2011 Lenuma Red (Douro),180,107,751,12189,442,93,65.0
4,70711,Quinta dos Avidagos 2012 Premium Red (Douro),180,107,751,12189,442,87,29.0


# STEP 3: LOAD

In [20]:
# Create connection
connection_string = "postgres:postgres@localhost:5432/test_wine_db"
engine = create_engine(f'postgresql://{connection_string}')

In [27]:
# Load all 5 dataframes into their respective tables
df_countries.to_sql(name='countries', con=engine, if_exists='append', index=False)
df_provinces.to_sql(name='provinces', con=engine, if_exists='append', index=False)
df_regions.to_sql(name='regions', con=engine, if_exists='append', index=False)
df_wineries.to_sql(name='wineries', con=engine, if_exists='append', index=False)
df_wine_types.to_sql(name='wine_types', con=engine, if_exists='append', index=False)
df_wine_final.to_sql(name='wines', con=engine, if_exists='append', index=False)

In [28]:
query_str = open('SQL_Files/test_query.sql')
query_text = ""
for text in query_str:
    query_text = query_text + text
      
print(query_text)
df_query = pd.read_sql_query(query_text, con=engine)

df_query

select w.wine_name "Wine", wt.wine_type "Type", ws.winery_name "Winery", c.country_name "Country", 
	p.province_name "Province",  r.region_name "Region", p.avg_f "Average Temperature", w.price "Price", w.rating "Rating"

from wines w 
	inner join countries c on w.country_id = c.country_id
	inner join provinces p on w.province_id = p.province_id
	inner join regions r on w.region_id = r.region_id
	inner join wineries ws on w.winery_id = ws. winery_id
	inner join wine_types wt on w.wine_type_id = wt.wine_type_id
	
where c.country_name = 'United States'
	




,Wine,Type,Winery,Country,Province,Region,Average Temperature,Price,Rating
0,Cupcake 2016 Rosé (California),Rosé,Cupcake,United States,California,California,59.4,13.0,89.0
1,Peju 2015 Rosé (Napa Valley),Rosé,Peju,United States,California,Napa Valley,59.4,25.0,90.0
2,Pino 2014 Pinot Gris (Oregon),Pinot Gris,Pino,United States,Oregon,Oregon,48.4,18.0,87.0
3,Pino 2015 Pinot Gris (Oregon),Pinot Gris,Pino,United States,Oregon,Oregon,48.4,18.0,87.0
4,Koenig Vineyards 2012 55% Syrah-45% Merlot Dry...,Rosé,Koenig Vineyards,United States,Idaho,No region,44.4,15.0,86.0
...,...,...,...,...,...,...,...,...,...
54260,Haak 2009 Light Madeira Jacquez (Texas),Jacquez,Haak,United States,Texas,Texas,64.8,40.0,82.0
54261,Haak 2011 Madeira Jacquez (Texas),Jacquez,Haak,United States,Texas,Texas,64.8,40.0,87.0
54262,McPherson 2015 Timmons Ranch Piquepoul Blanc (...,Piquepoul Blanc,McPherson,United States,Texas,Texas,64.8,13.0,88.0
54263,Les Bourgeois 2010 Chardonel (Missouri),Chardonel,Les Bourgeois,United States,Missouri,Missouri,54.5,11.0,83.0
